In [ ]:
import mobi
import html2text
import regex as re
import pandas as pd

In [ ]:
def link_getter(i):
    values = {}
    if i in range(411,457):
        values['index'] = i-411 + 6
        values['year'] = 2021
    elif i in range(457,508):
        values['index'] = i-457 + 1
        values['year'] = 2022
    else:
        values['index'] = i-508 + 1
        values['year'] = 2023

    if i > 524:
        values['index']-=1    #trick

    return f"https://tygodnik.dorzeczy.pl/archiwum/{i}/dorzeczy-{values['index']}-{values['year']}.html"

link_getter(456)

In [ ]:
def prepare_title(title):
    title = re.sub(r'\?', r'\?', title)
    title = re.sub(r'\.', r'\.', title)
    title = re.sub(r'\(', r'\(', title)
    title = re.sub(r'\)', r'\)', title)
    title = re.sub(r'\+', r'\+', title)
    title = re.sub(r'\n', r' ', title)

    return title

def author_extract(text):
    match = re.search("(?<=\\n\\n).+(?=\s*!\[\])", text)
    if match != None:
        match = match[0].strip()
    return match

def text_preprocessing(text):
    text = re.sub(r"[^\*]+", '',text, count = 1)
    text = text.replace('*', ' ')
    return text

In [ ]:
def pipeline(txt):

    table_of_contents = re.split( "\*\*Table of Contents\*\*",txt)[1]
    print(re.findall("\d+(?=\\\.\s)", table_of_contents)[-1], end=" ")
    titles = re.split("\n\n\d+\\\.\s", table_of_contents)[1:]
    titles[-1] = titles[-1].strip()
    articles = []
    for i in range(len(titles)-1):
        title_1 = prepare_title(titles[i])
        title_2 = prepare_title(titles[i+1])
        
        if(len(title_1)>50):
            split = re.split('\*\*'+title_1[:50]+'[^\*]+\*\*', txt)
        else:
            split = re.split('\*\*'+title_1+'\*\*', txt)
        
        txt_1 = " ".join(split[1:])

        if(len(title_2)>50):
            split = re.split('\*\*'+title_2[:50]+'[^\*]+\*\*', txt_1)
        else:
            split = re.split('\*\*'+ title_2 + '\*\*', txt_1)
    
        articles.append((split[0], titles[i]))
 

    title_last = prepare_title(titles[-1])
    
    if(len(title_last)>50):
        split = re.split('\*\*'+title_last[:50]+'[^\*]+\*\*', txt)
    else:
        split = re.split('\*\*'+title_last+'\*\*', txt)
    
    if len(split)!=2: print(title_last + " ??? ", end="")
    txt_1 = split[1]
    
    splits = ['Na łamach .*Do Rzeczy.* również:','\*\*Table of Contents\*\*']
    for split in splits:
        txt_splited = re.split(split, txt_1)
        if len(txt_splited) == 2:
            articles.append((txt_splited[0], titles[-1]))
            break

    print(len(articles))
    df = pd.DataFrame(articles, columns=["article_raw", "title"])
    df['author'] = df['article_raw'].apply(author_extract)
    df['text'] = df['article_raw'].apply(text_preprocessing)
    return df[['text','author','title']]

In [ ]:
#df_list = []
#for i in range(459,479):
#    filename = f"Mobi2022/{i}.mobi"
#    print(filename, end="\t")
#    try:
#        tempdir, filepath = mobi.extract(filename)
#    except:
#        print("File not found")
#        continue
#    file = open(filepath, "r")
#
#    content=file.read()
#    txt = html2text.html2text(content)
#        
#    df = pipeline(txt)
#    df['magazine_nr'] = i
#    df['year'] = 2022
#    df['link'] = link_getter(i)
#    df_list.append(df)


In [ ]:
#df_22 = pd.concat(df_list)
#len(df_22)

In [ ]:
#df_list = []
#for i in range(455,457):
#    filename = f"Mobi2021/{i}.mobi"
#    print(filename, end="\t")     
#
#    if i == 437:
#        with open(str(i)+".txt",'r') as f:
#            txt = f.read()
#
#            df_list.append(pipeline(txt))
#            continue
#    try:
#        tempdir, filepath = mobi.extract(filename)
#    except:
#        print("Doesnt exist")
#        continue
#    file = open(filepath, "r")
#
#    try:
#        content=file.read()
#        txt = html2text.html2text(content)
#        
#    except UnicodeDecodeError:
#        print("Unable to read")
#        continue
#        
#    df = pipeline(txt)
#    df['magazine_nr'] = i
#    df['year'] = 2021
#    df['link'] = link_getter(i)
#    df_list.append(df)

In [ ]:
#df_21 = pd.concat(df_list)
#len(df_21)

In [ ]:
from os import walk
df_list = []
files = []
for (dirpath, dirnames, filenames) in walk("Mobi2023"):
    files.extend(filenames)
    
num = 508

In [ ]:
for filename in files:

    print(filename +" "+ str(num), end="\t")  

    if(filename =="dorzeczy-24-2023-.mobi"):

        filename = filename.replace("mobi", "txt")
        with open(filename, "r")  as file:
            txt = file.read()

    else:

        tempdir, filepath = mobi.extract("Mobi2023/" + filename)

        file = open(filepath, "r")

        try:
            content=file.read()
            txt = html2text.html2text(content)
            with open("mobi.txt", "w") as f:
                f.write(txt)
        except UnicodeDecodeError:
            print("Unable to read")
            continue
    
    df = pipeline(txt)
    if (num == 527): num+=1
    df['magazine_nr'] = num
    df['link'] = link_getter(num)
    df['year'] = 2023
    df_list.append(df)
    num += 1
    

In [ ]:
df_23 = pd.concat(df_list)
len(df_23)

In [ ]:
df_all = pd.concat([df_23])

In [ ]:
len(df_all)

In [ ]:
df_all.to_csv("from_mobi.csv")